# Kaggle Submission

In [7]:
# General tools
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.display import display
pd.options.display.max_columns = None
from pprint import pprint
import io

# Classification
from sklearn.metrics import f1_score
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [8]:
# Google COLAB file import (TRAIN)
# from google.colab import files
# trainUpload = files.upload()

In [9]:
# Google COLAB file import (TEST)
# from google.colab import files
# testUpload = files.upload()

# DataFrame Setup

## Helper Functions

In [10]:
# NO RENT PREDICTED / NO TRANSFORMATIONS
def dataframe_generator(data, rent=False):

    # Load dataframe
    df = pd.read_csv(data)


    #_______________________________
    # RENT
    #_______________________________

    if rent == False:
        df.drop('v2a1', axis=1, inplace=True)


    #_______________________________
    # NUMBER OF TABLETS
    #_______________________________

    df.v18q1.fillna(0, inplace=True)


    #_______________________________
    # YEARS BEHIND IN SCHOOL
    #_______________________________

    df.drop("rez_esc", axis=1, inplace=True)


    #_______________________________
    # MIXED CATEGORY VARIABLES
    #_______________________________

    df.drop(columns=['edjefe', 'edjefa', 'dependency'], inplace=True)


    #_______________________________
    # EDUCATION
    #_______________________________

    # Find missing education value ids
    missing_ids = df[df['meaneduc'].isna()]['meaneduc'].keys()

    # Adult (18+) education by household
    educ_by_household = df[df.age >= 18].groupby('idhogar')['escolari'].mean()

    # Iterate over missing values and set them to correct value
    for i in missing_ids:

        household = df.loc[i, 'idhogar']

        # Update 'meaneduc'
        try:
            df.loc[i, 'meaneduc'] = educ_by_household[household]
        except:
            df.loc[i, 'meaneduc'] = 0

        # Update 'SQBmeaned'
        try:
            df.loc[i, 'SQBmeaned'] = educ_by_household[household] ** 2
        except:
            df.loc[i, 'SQBmeaned'] = 0


    #_______________________________
    # DROPPING IDENTIFIERS
    #_______________________________

    # df.drop(columns=['Id', 'idhogar'], inplace=True)


    return df



# YES RENT PREDICTED / NO TRANSFORMATIONS
def dataframe_generator_rent(data):
    
    #_______________________________
    # DATAFRAME SETUP
    #_______________________________
    
    # Setting up new dataframe (including rent data)
    df_rent = dataframe_generator(data, rent=True)
    
    # Remove missing values for target (rent)
    df_rent_predict = df_rent.dropna()

    
    #_______________________________
    # CLASSIFICATION SETUP
    #_______________________________
    
    # Partition explanatory and response variables
    X = df_rent_predict.drop(columns=['v2a1', 'Id', 'idhogar'])
    y = df_rent_predict['v2a1']

    # Split into training and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=12345)
    
    
    #_______________________________
    # CLASSIFICATION 
    # (using random forest because it consistently gave highest score)
    #_______________________________
    
    # Random Forest
    clf = RandomForestRegressor()
    clf.fit(X_train, y_train)
    # print(clf.score(X_test, y_test))
    
    
    #_______________________________
    # FILL NAN USING PREDICTED VALUES FROM MODEL
    #_______________________________
    
    # Prepare data to fill in predicted values for rent
    df_rent_nan = df_rent[df_rent.v2a1.isna()]
    
    # Predict using model
    rent_pred = clf.predict(df_rent_nan.drop(columns=['v2a1', 'Id', 'idhogar']))
    
    # Fill NaN
    df_rent_nan['v2a1'] = pd.DataFrame(rent_pred).values
    
    # Update full dataframe
    df_rent[df_rent.v2a1.isna()] = df_rent_nan
    
    
    return df_rent



# YES RENT PREDICTED / YES TRANSFORMATIONS
def dataframe_generator_trans(data):
    
    # Top Features
    top_features = ['v2a1', 'meaneduc', 'SQBedjefe', 'overcrowding', 'SQBdependency', 'age', 'rooms', 'qmobilephone']

    # Best subset
    winner = \
            [['SQ_SQBedjefe',
              'LOG_qmobilephone',
              'SQ_v2a1',
              'SQBdependency',
              'SQBedjefe',
              'meaneduc',
              'qmobilephone',
              'rooms',
              'LOG_meaneduc',
              'SQ_qmobilephone',
              'v2a1',
              'SQ_overcrowding',
              'LOG_SQBdependency'],
             13,
             0.9257322175732218,
             0.8887133182436542]
            
    # Create rent-inclusive dataframe
    df_rent = dataframe_generator_rent(data)
    
    # Create transformed dataframe
    df_trans = df_rent.copy(deep=True)
    df_trans.drop(columns=top_features, inplace=True)

    for feature in winner[0]:
        if "SQ_" in feature:
            col = feature.split("SQ_")[1]
            df_trans[feature] = df_rent[col] ** 2

        elif "LOG_" in feature:
            col = feature.split("LOG_")[1]
            df_trans[feature] = df_rent[col].apply(lambda x: np.log(x) if x!=0 else x)

        else:
            col = feature
            df_trans[feature] = df_rent[col]
            
    return df_trans

In [11]:
# Google Colab Approach
# df_rent = pd.read_csv(io.BytesIO(trainUpload['train.csv']))

df = dataframe_generator_trans("../input/train.csv")
df.head()

/home/nemosaic/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Id,hacdor,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,techootro,cielorazo,abastaguadentro,abastaguafuera,abastaguano,public,planpri,noelec,coopele,sanitario1,sanitario2,sanitario3,sanitario5,sanitario6,energcocinar1,energcocinar2,energcocinar3,energcocinar4,elimbasu1,elimbasu2,elimbasu3,elimbasu4,elimbasu5,elimbasu6,epared1,epared2,epared3,etecho1,etecho2,etecho3,eviv1,eviv2,eviv3,dis,male,female,estadocivil1,estadocivil2,estadocivil3,estadocivil4,estadocivil5,estadocivil6,estadocivil7,parentesco1,parentesco2,parentesco3,parentesco4,parentesco5,parentesco6,parentesco7,parentesco8,parentesco9,parentesco10,parentesco11,parentesco12,idhogar,hogar_nin,hogar_adul,hogar_mayor,hogar_total,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,SQBescolari,SQBage,SQBhogar_total,SQBhogar_nin,SQBovercrowding,SQBmeaned,agesq,Target,SQ_SQBedjefe,LOG_qmobilephone,SQ_v2a1,SQBdependency,SQBedjefe,meaneduc,qmobilephone,rooms,LOG_meaneduc,SQ_qmobilephone,v2a1,SQ_overcrowding,LOG_SQBdependency
0,ID_279628684,0,0,1,1,0,0.0,0,1,1,0,0,0,0,1,1,1,1,10,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,21eb7fcc1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,100,1849,1,0,1.000000,100.0,1849,4,10000,0.000000,3.610000e+10,0.0,100,10.0,1,3,2.302585,1,190000.0,1.000000,0.000000
1,ID_f29eb3ddd,0,0,1,1,1,1.0,0,1,1,0,0,0,0,1,1,1,1,12,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0e5d7a658,0,1,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,144,4489,1,0,1.000000,144.0,4489,4,20736,0.000000,1.822500e+10,64.0,144,12.0,1,4,2.484907,1,135000.0,1.000000,4.158883
2,ID_68de51c94,0,0,1,1,0,0.0,0,0,0,0,1,1,0,1,1,1,1,11,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2c7317ea8,0,1,1,1,0,0,0,0,1,0,0,0,0,2,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,121,8464,1,0,0.250000,121.0,8464,4,0,0.000000,7.491169e+10,64.0,0,11.0,0,8,2.397895,0,273700.0,0.250000,4.158883
3,ID_d671db89c,0,0,1,1,1,1.0,0,2,2,1,1,2,1,3,4,4,4,9,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,2b58d945f,2,2,0,4,0,0,0,1,0,0,0,0,0,3,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,81,289,16,4,1.777778,121.0,289,4,14641,1.098612,3.240000e+10,1.0,121,11.0,3,5,2.397895,9,180000.0,1.777778,0.000000
4,ID_d56d6f5f5,0,0,1,1,1,1.0,0,2,2,1,1,2,1,3,4,4,4,11,4,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2b58d945f,2,2,0,4,0,0,0,0,1,0,0,0,0,3,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,121,1369,16,4,1.777778,121.0,1369,4,14641,1.098612,3.240000e+10,1.0,121,11.0,3,5,2.397895,9,180000.0,1.777778,0.000000


In [12]:
# Partition explanatory and response variables
X = df.drop(columns=['Target', 'Id', 'idhogar']) # Dropping identifiers and target
print(X.shape)

y = df['Target']
print(y.shape)

(9557, 141)
(9557,)


# Classification

In [13]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Train Best Random Model

In [14]:
# # best_random = RandomForestClassifier(bootstrap=False, max_depth=50, max_features='auto', 
# #                                      min_samples_leaf=1, min_samples_split=2, n_estimators=1000)

# best_random = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
#             max_depth=None, max_features='sqrt', max_leaf_nodes=None,
#             min_impurity_decrease=0.0, min_impurity_split=None,
#             min_samples_leaf=1, min_samples_split=2,
#             min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=None,
#             oob_score=False, random_state=None, verbose=0,
#             warm_start=False)

# best_random.fit(X_trim_train,y)

## Train Best GridSearch Model

In [15]:
best_grid_search = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=50, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

best_grid_search.fit(X, y)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## Test Data Setup

In [16]:
# Google Colab Approach
# df_test = pd.read_csv(io.BytesIO(testUpload['test.csv']))

df_test = dataframe_generator_trans("../input/test.csv")
df_test.head()

/home/nemosaic/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Id,hacdor,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,r4h3,r4m1,r4m2,r4m3,r4t1,r4t2,r4t3,tamhog,tamviv,escolari,hhsize,paredblolad,paredzocalo,paredpreb,pareddes,paredmad,paredzinc,paredfibras,paredother,pisomoscer,pisocemento,pisoother,pisonatur,pisonotiene,pisomadera,techozinc,techoentrepiso,techocane,techootro,cielorazo,abastaguadentro,abastaguafuera,abastaguano,public,planpri,noelec,coopele,sanitario1,sanitario2,sanitario3,sanitario5,sanitario6,energcocinar1,energcocinar2,energcocinar3,energcocinar4,elimbasu1,elimbasu2,elimbasu3,elimbasu4,elimbasu5,elimbasu6,epared1,epared2,epared3,etecho1,etecho2,etecho3,eviv1,eviv2,eviv3,dis,male,female,estadocivil1,estadocivil2,estadocivil3,estadocivil4,estadocivil5,estadocivil6,estadocivil7,parentesco1,parentesco2,parentesco3,parentesco4,parentesco5,parentesco6,parentesco7,parentesco8,parentesco9,parentesco10,parentesco11,parentesco12,idhogar,hogar_nin,hogar_adul,hogar_mayor,hogar_total,instlevel1,instlevel2,instlevel3,instlevel4,instlevel5,instlevel6,instlevel7,instlevel8,instlevel9,bedrooms,tipovivi1,tipovivi2,tipovivi3,tipovivi4,tipovivi5,computer,television,mobilephone,lugar1,lugar2,lugar3,lugar4,lugar5,lugar6,area1,area2,SQBescolari,SQBage,SQBhogar_total,SQBhogar_nin,SQBovercrowding,SQBmeaned,agesq,SQ_SQBedjefe,LOG_qmobilephone,SQ_v2a1,SQBdependency,SQBedjefe,meaneduc,qmobilephone,rooms,LOG_meaneduc,SQ_qmobilephone,v2a1,SQ_overcrowding,LOG_SQBdependency
0,ID_2f6873615,0,0,1,1,0,0.0,1,1,2,0,1,1,1,2,3,3,3,0,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,72958b30c,1,2,0,3,1,0,0,0,0,0,0,0,0,2,1,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,16,9,1,2.25,272.25,16,0,0.693147,6.327941e+10,0.25,0,16.5,2,5,2.803360,4,251554.0,2.25,-1.386294
1,ID_1c78846d2,0,0,1,1,0,0.0,1,1,2,0,1,1,1,2,3,3,3,16,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,72958b30c,1,2,0,3,0,0,0,0,0,0,0,1,0,2,1,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,256,1681,9,1,2.25,272.25,1681,0,0.693147,8.529554e+10,0.25,0,16.5,2,5,2.803360,4,292054.0,2.25,-1.386294
2,ID_e5442cf6a,0,0,1,1,0,0.0,1,1,2,0,1,1,1,2,3,3,3,17,3,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,72958b30c,1,2,0,3,0,0,0,0,0,0,0,0,1,2,1,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,289,1681,9,1,2.25,272.25,1681,0,0.693147,8.459605e+10,0.25,0,16.5,2,5,2.803360,4,290854.0,2.25,-1.386294
3,ID_a8db26a79,0,0,1,1,1,1.0,0,1,1,0,0,0,0,1,1,1,1,16,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5b598fbc9,0,1,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,256,3481,1,0,1.00,256.00,3481,65536,0.693147,2.479648e+12,0.00,256,16.0,2,14,2.772589,4,1574689.7,1.00,0.000000
4,ID_a62966799,0,0,1,1,1,1.0,0,0,0,0,1,1,0,1,1,1,1,11,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1e2fc704e,1,0,0,1,0,0,0,0,1,0,0,0,0,2,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,121,324,1,1,0.25,121.00,324,0,0.000000,3.062500e+10,64.00,0,11.0,1,4,2.397895,1,175000.0,0.25,4.158883


In [17]:
# Partition explanatory variables
Id = df_test.Id
X_test = df_test.drop(columns=['Id','idhogar']) # Dropping identifiers

In [18]:
# GridSearch
# Target = best_random.predict(X_test)
Target = best_grid_search.predict(X_test)

In [19]:
Id = pd.Series(Id) 

In [20]:
Target = pd.Series(Target)
Target.head(10)

0    4
1    4
2    4
3    4
4    4
5    4
6    4
7    4
8    4
9    4
dtype: int64

In [21]:
kaggle_submit = pd.concat([Id, Target], axis=1)

In [22]:
kaggle_submit.columns = ['Id', 'Target']

In [1]:
kaggle_submit.to_csv("submission.csv", index=False)